<a href="https://colab.research.google.com/github/safranco8/entregables-grupo-31/blob/main/AprendizajeSupervisado/TP_supervisado_correcciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importación de Librerias

In [ ]:
!pip install catboost

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report

#Modelos
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier

#otros
from sklearn.ensemble import VotingClassifier
import pandas_profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


# Lectura de Datasets

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/safranco8/entregables-grupo-31/main/AprendizajeSupervisado/train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/safranco8/entregables-grupo-31/main/AprendizajeSupervisado/test.csv')

print(train_df.shape)
print(train_df.info())
print(train_df.describe())

(7823, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7823 entries, 0 to 7822
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   7823 non-null   int64  
 1   HomePlanet    7644 non-null   object 
 2   CryoSleep     7627 non-null   object 
 3   Cabin         7639 non-null   object 
 4   Destination   7663 non-null   object 
 5   Age           7661 non-null   float64
 6   VIP           7639 non-null   object 
 7   RoomService   7669 non-null   float64
 8   FoodCourt     7668 non-null   float64
 9   ShoppingMall  7635 non-null   float64
 10  Spa           7665 non-null   float64
 11  VRDeck        7654 non-null   float64
 12  Name          7639 non-null   object 
 13  Transported   7823 non-null   int64  
dtypes: float64(6), int64(2), object(6)
memory usage: 855.8+ KB
None
       PassengerId          Age   RoomService     FoodCourt  ShoppingMall  \
count  7823.000000  7661.000000   7669.000000   

# Exploración de Datos

In [ ]:
pandas_profiling.ProfileReport(train_df)

TypeError: ignored


# Escalado e ingenieria de características

In [ ]:
#Datasets de entrada y salida

train_y = train_df.Transported
train_X = train_df.drop('Transported',axis=1)

#NO existe "Transported" en test (objetivo)
#test_y = test_df.Transported
test_X = test_df #.drop('Transported',axis=1)

train_X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,3430,Europa,False,C/123/P,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,Azhah Tronsive
1,669,Mars,True,F/126/S,NaN,18.0,False,0.0,0.0,0.0,0.0,0.0,Roswal Sha
2,8430,Europa,True,B/343/S,TRAPPIST-1e,33.0,NaN,0.0,0.0,0.0,0.0,0.0,Shaiam Tinganking
3,6463,Earth,False,F/1420/P,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,Landa Grifford
4,85,Earth,False,G/9/P,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,Gracy Dunnisey
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7818,6528,Mars,False,F/1319/S,TRAPPIST-1e,12.0,NaN,0.0,0.0,0.0,0.0,0.0,Risps Hacle
7819,2693,Earth,False,G/459/S,TRAPPIST-1e,44.0,False,0.0,0.0,744.0,62.0,3.0,Gabrie Reenez
7820,8076,Mars,True,F/1772/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Panzel Ruity
7821,3829,Earth,False,E/253/P,TRAPPIST-1e,15.0,False,0.0,840.0,123.0,0.0,20.0,Belly Weavesend


In [ ]:
#(TRAIN 80% / VALIDATION 20%) - TEST
x_train, x_val, y_train, y_val = train_test_split(train_X, train_y, train_size=0.8, random_state = 0)

In [ ]:
#No necesitamos la columna de nombre
x_train.drop(['Name'], axis=1, inplace=True)
x_val.drop(['Name'], axis=1, inplace=True)
test_X.drop(['Name'], axis=1, inplace=True)

In [ ]:
# Replace NaN's with outliers for now (so we can split feature)
x_train['Cabin'].fillna('Z/9999/Z', inplace=True)
x_val['Cabin'].fillna('Z/9999/Z', inplace=True)
test_X['Cabin'].fillna('Z/9999/Z', inplace=True)

# New features - training set
x_train['Cabin_deck'] = x_train['Cabin'].apply(lambda x: x.split('/')[0])
x_train['Cabin_number'] = x_train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
x_train['Cabin_side'] = x_train['Cabin'].apply(lambda x: x.split('/')[2])

# New features - validation set
x_val['Cabin_deck'] = x_val['Cabin'].apply(lambda x: x.split('/')[0])
x_val['Cabin_number'] = x_val['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
x_val['Cabin_side'] = x_val['Cabin'].apply(lambda x: x.split('/')[2])

# New features - test set
test_X['Cabin_deck'] = test_X['Cabin'].apply(lambda x: x.split('/')[0])
test_X['Cabin_number'] = test_X['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
test_X['Cabin_side'] = test_X['Cabin'].apply(lambda x: x.split('/')[2])

# Put Nan's back in (we will fill these later)
x_train.loc[x_train['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
x_train.loc[x_train['Cabin_number']==9999, 'Cabin_number']=np.nan
x_train.loc[x_train['Cabin_side']=='Z', 'Cabin_side']=np.nan

x_val.loc[x_val['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
x_val.loc[x_val['Cabin_number']==9999, 'Cabin_number']=np.nan
x_val.loc[x_val['Cabin_side']=='Z', 'Cabin_side']=np.nan

test_X.loc[test_X['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
test_X.loc[test_X['Cabin_number']==9999, 'Cabin_number']=np.nan
test_X.loc[test_X['Cabin_side']=='Z', 'Cabin_side']=np.nan

# Drop Cabin (we don't need it anymore)
x_train.drop('Cabin', axis=1, inplace=True)
x_val.drop('Cabin', axis=1, inplace=True)
test_X.drop('Cabin', axis=1, inplace=True)

In [ ]:
#Imputacion de faltantes

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]

imputer = SimpleImputer(strategy="median")

x_train[imputer_cols] = imputer.fit_transform(x_train[imputer_cols])
x_val[imputer_cols] = imputer.fit_transform(x_val[imputer_cols])
test_X[imputer_cols] = imputer.fit_transform(test_X[imputer_cols])

imputer_cols_cat = ["Cabin_deck","Cabin_number","Cabin_side"]

imputer_cat = SimpleImputer(strategy="most_frequent")

x_train[imputer_cols_cat] = imputer_cat.fit_transform(x_train[imputer_cols_cat])
x_val[imputer_cols_cat] = imputer_cat.fit_transform(x_val[imputer_cols_cat])
test_X[imputer_cols_cat] = imputer_cat.fit_transform(test_X[imputer_cols_cat])



Algún análisis exploratorio que permita tomar decisiones sobre las variabes a usar, transformaciones de variables, tratamiento de los datos faltantes, etc. etc.

In [ ]:
# elijo y transformo algunas variables (en su análisis fundamente sus decisiones)

#Saco las columnas categoricas que tienen muchos valores distintos (no hay clases
#para la toma de decisión)

for var in x_train:
    if x_train[var].dtype == 'object':
        unique_cases = x_train[var].unique()
        if len(unique_cases) > 10:
            print("BORRO columna '", var, "' con", len(unique_cases), "valores distintos (en train).")
            x_train.drop(var, inplace=True, axis=1)
            x_val.drop(var, inplace=True, axis=1)
            test_X.drop(var, inplace=True, axis=1)


BORRO columna ' Cabin_number ' con 1689 valores distintos (en train).


In [ ]:
#No necesitamos el Id de Pasajero 
passengerId = x_train.PassengerId
x_train.drop('PassengerId',axis=1,inplace=True)

passengerId = x_val.PassengerId
x_val.drop('PassengerId',axis=1,inplace=True)

test_id = test_X.PassengerId #Lo guardo para el final
test_X.drop('PassengerId',axis=1,inplace=True)


In [ ]:
#Encoding categoricas

label_cols = ["HomePlanet", "CryoSleep", "Destination" ,"VIP","Cabin_deck","Cabin_side"]

for col in label_cols:

  # dataframe de variables dummy para cada categoría
  uniques = x_train[col].unique()
  uniques_df = pd.DataFrame(uniques, columns=[col])
  dummy_df = pd.get_dummies(uniques_df, columns=[col], prefix=[col + "_type"])
  uniques_df = uniques_df.join(dummy_df)
  
  x_train = x_train.join(uniques_df.set_index(col), on=col)
  x_val = x_val.join(uniques_df.set_index(col), on=col)
  test_X = test_X.join(uniques_df.set_index(col), on=col)

  # Por si hay algún valor nuevo en val o test que no aparezca en train
  for dummy_col in uniques_df.columns.values: 
    x_val[dummy_col].fillna(0,inplace=True)
    test_X[dummy_col].fillna(0,inplace=True)

  # Descarto las columnas originales
  x_train.drop(col,axis=1,inplace=True)
  x_val.drop(col,axis=1,inplace=True)
  test_X.drop(col,axis=1,inplace=True)


In [ ]:
# Estandarizo todo el dataset

scaler = StandardScaler()    
  
train_columns = x_train.columns
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=train_columns)

val_columns = x_val.columns
x_val = scaler.transform(x_val)
x_val = pd.DataFrame(x_val, columns=val_columns)

test_columns = test_X.columns
test_X = scaler.transform(test_X)
test_X = pd.DataFrame(test_X, columns=test_columns)


In [ ]:
x_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_type_Earth,HomePlanet_type_Europa,HomePlanet_type_Mars,CryoSleep_type_False,...,Cabin_deck_type_A,Cabin_deck_type_B,Cabin_deck_type_C,Cabin_deck_type_D,Cabin_deck_type_E,Cabin_deck_type_F,Cabin_deck_type_G,Cabin_deck_type_T,Cabin_side_type_P,Cabin_side_type_S
0,0.982047,-0.337865,0.022606,-0.274006,-0.260866,1.982360,-1.047466,1.728001,-0.506334,0.787125,...,-0.178387,3.186704,-0.311031,-0.240436,-0.335755,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
1,-0.207547,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,1.728001,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,4.159116,-0.335755,-0.718323,-0.646345,-0.01788,1.029189,-1.029189
2,-1.956949,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,-0.578703,1.974981,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,1.029189,-1.029189
3,0.142334,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,-0.578703,1.974981,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,2.978366,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
4,0.072358,0.012552,-0.280474,-0.274006,-0.027625,-0.233801,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,-0.971639,0.971639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6253,0.142334,-0.336348,-0.280474,-0.270751,0.254138,-0.257377,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,1.029189,-1.029189
6254,0.842095,-0.327246,-0.280474,0.209429,-0.168932,0.069969,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,-0.971639,0.971639
6255,-0.207547,2.497326,-0.280474,-0.052635,-0.260866,-0.257377,-1.047466,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,1.029189,-1.029189
6256,-0.627403,-0.299941,-0.280474,1.021665,-0.223412,-0.217479,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,1.029189,-1.029189


In [ ]:
x_val

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_type_Earth,HomePlanet_type_Europa,HomePlanet_type_Mars,CryoSleep_type_False,...,Cabin_deck_type_A,Cabin_deck_type_B,Cabin_deck_type_C,Cabin_deck_type_D,Cabin_deck_type_E,Cabin_deck_type_F,Cabin_deck_type_G,Cabin_deck_type_T,Cabin_side_type_P,Cabin_side_type_S
0,0.212310,-0.337865,0.946814,-0.252846,0.633794,-0.257377,-1.047466,1.728001,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,4.159116,-0.335755,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
1,0.072358,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,-0.578703,1.974981,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,1.029189,-1.029189
2,1.052023,-0.333314,-0.280474,-0.220291,0.393743,-0.257377,-1.047466,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
3,-0.137570,-0.073914,0.126751,-0.274006,-0.260866,-0.257377,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
4,1.891736,-0.337865,0.205327,-0.254473,-0.258313,-0.225640,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,1.029189,-1.029189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,-0.137570,-0.337865,-0.274862,-0.274006,-0.226817,6.653164,-1.047466,1.728001,-0.506334,0.787125,...,-0.178387,-0.313804,3.215114,-0.240436,-0.335755,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
1561,-0.067594,-0.337865,-0.280474,-0.274006,0.314577,-0.144937,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
1562,-0.557427,-0.183135,-0.274862,-0.239824,0.134112,-0.163073,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
1563,0.702143,-0.334831,-0.280474,-0.274006,-0.259164,0.456255,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,2.978366,-0.718323,-0.646345,-0.01788,-0.971639,0.971639


In [ ]:
test_X

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_type_Earth,HomePlanet_type_Europa,HomePlanet_type_Mars,CryoSleep_type_False,...,Cabin_deck_type_A,Cabin_deck_type_B,Cabin_deck_type_C,Cabin_deck_type_D,Cabin_deck_type_E,Cabin_deck_type_F,Cabin_deck_type_G,Cabin_deck_type_T,Cabin_side_type_P,Cabin_side_type_S
0,0.352262,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,1.728001,-0.506334,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
1,-1.117236,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,0.954685,-0.578703,-0.506334,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,-0.971639,0.971639
2,1.052023,-0.304492,-0.279851,-0.274006,0.474611,-0.257377,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,1.029189,-1.029189
3,2.381569,-0.337865,-0.280474,-0.143788,-0.260866,1.848157,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,2.978366,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
4,-1.327164,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,-1.047466,1.728001,-0.506334,0.787125,...,-0.178387,3.186704,-0.311031,-0.240436,-0.335755,-0.718323,-0.646345,-0.01788,-0.971639,0.971639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,1.751784,0.687597,-0.280474,0.101999,0.061756,-0.257377,-1.047466,-0.578703,1.974981,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639
866,2.031688,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,0.954685,-0.578703,-0.506334,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,-0.971639,0.971639
867,-0.767355,-0.337865,-0.280474,-0.274006,-0.260866,-0.257377,0.954685,-0.578703,-0.506334,-1.270447,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,-0.718323,1.547162,-0.01788,1.029189,-1.029189
868,-0.767355,0.822607,-0.280474,-0.252846,-0.260866,-0.257377,0.954685,-0.578703,-0.506334,0.787125,...,-0.178387,-0.313804,-0.311031,-0.240436,-0.335755,1.392131,-0.646345,-0.01788,-0.971639,0.971639


# Creo los conjuntos de datos de entrenamiento y pruebas

In [ ]:
#(TRAIN 80% / VALIDATION 20%) - TEST
# x_train, x_val, y_train, y_val = train_test_split(train_X, train_y, train_size=0.8, random_state = 0)


# Planteo modelos
Tienen que elegir los 2 mejores modelos así que prueben con varios.


In [ ]:
clfs = [DecisionTreeClassifier(random_state=0), 
        CatBoostClassifier(verbose=0),
        LGBMClassifier(random_state=0), 
        RidgeClassifier(random_state=0),
        XGBClassifier(random_state=0),
        MLPClassifier(random_state=0),
        KNeighborsClassifier(),
        RandomForestClassifier(random_state=0)
       ]
names = ['DecisionTreeClassifier', 
         'CatBoostClassifier',
         'LGBMClassifier', 
         'RidgeClassifier',
         'XGBClassifier', 
         'MLPClassifier', 
         'KNeighborsClassifier',
         'RandomForestClassifier'
        ]

"""
LGBM = LGBMClassifier(**LGBM_param)
RD = RidgeClassifier(**RIDGE_param)
XGB =XGBClassifier(**XGB_param)
MLP = MLPClassifier(**MLP_params)
KNN = KNeighborsClassifier(**KNN_param)
"""

trained_models = []
accuracy_models = []
for clf, name in zip(clfs,names):
    print("=====", name, "=====")
    clf.fit(x_train, y_train)
    train_predictions = clf.predict(x_train)
    accuracy = accuracy_score(y_train, train_predictions)
    print(f"Accuracy train {name}: %.2f%%" % (accuracy * 100.0))

    train_predictions = clf.predict(x_val)
    accuracy = accuracy_score(y_val, train_predictions)
    print(f"Accuracy validation {name}: %.2f%%" % (accuracy * 100.0))
    #plot_confusion_matrix(clf,x_test,y_test)
    trained_models.append(clf)
    accuracy_models.append(accuracy*100)

===== DecisionTreeClassifier =====
Accuracy train DecisionTreeClassifier: 95.51%
Accuracy validation DecisionTreeClassifier: 74.12%
===== CatBoostClassifier =====
Accuracy train CatBoostClassifier: 85.83%
Accuracy validation CatBoostClassifier: 81.02%
===== LGBMClassifier =====
Accuracy train LGBMClassifier: 86.80%
Accuracy validation LGBMClassifier: 80.45%
===== RidgeClassifier =====
Accuracy train RidgeClassifier: 76.89%
Accuracy validation RidgeClassifier: 77.44%
===== XGBClassifier =====
Accuracy train XGBClassifier: 81.58%
Accuracy validation XGBClassifier: 81.28%
===== MLPClassifier =====


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy train MLPClassifier: 83.38%
Accuracy validation MLPClassifier: 81.41%
===== KNeighborsClassifier =====
Accuracy train KNeighborsClassifier: 83.64%
Accuracy validation KNeighborsClassifier: 77.83%
===== RandomForestClassifier =====
Accuracy train RandomForestClassifier: 95.51%
Accuracy validation RandomForestClassifier: 79.74%


### Algún tipo de ajuste de hiperparámetros de los modelos elegidos

#### Arbol de decisión

In [ ]:
search_grid = {'criterion': ('gini','entropy'),
               'splitter':("best","random"),
               'max_depth':(2, 5, 8, 10, 12, 20, 30),
               'min_samples_leaf': (1, 2, 4, 8, 10, 12, 15, 20),
               'max_features': ('sqrt', 'log2', None)}
grid = GridSearchCV(DecisionTreeClassifier(random_state=0), search_grid, cv=5, verbose=3)
grid.fit(x_train, y_train)
grid.best_params_



Fitting 5 folds for each of 672 candidates, totalling 3360 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=best;, score=0.726 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=best;, score=0.738 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=best;, score=0.730 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=best;, score=0.728 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=best;, score=0.748 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=random;, score=0.711 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, splitter=random;, score=0.727 total time=   0.0s
[CV 3/5] END criterion=gini, max_dep

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 10,
 'splitter': 'best'}

In [ ]:
# Best params: {'criterion': 'gini','max_depth': 10,'max_features': None,'min_samples_leaf': 10,'splitter': 'best'}
# clf = DecisionTreeClassifier(criterion='gini',max_depth=10, max_features=None, min_samples_leaf=10, splitter='best', random_state=0).fit(x_train, y_train)

clf = DecisionTreeClassifier(**grid.best_params_,random_state=0).fit(x_train, y_train)

train_predictions = clf.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train Arbol de decisión: %.2f%%" % (accuracy * 100.0))

val_predictions = clf.predict(x_val)
accuracy = accuracy_score(y_val, val_predictions)
print("Accuracy test Arbol de decisión: %.2f%%" % (accuracy * 100.0))
ConfusionMatrixDisplay.from_predictions(y_val, val_predictions)
plt.show()

print(classification_report(y_val, clf.predict(x_val)))

Accuracy train Arbol de decisión: 80.76%
Accuracy test Arbol de decisión: 79.17%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       778
           1       0.81      0.77      0.79       787

    accuracy                           0.79      1565
   macro avg       0.79      0.79      0.79      1565
weighted avg       0.79      0.79      0.79      1565



#### LGBM

In [ ]:
LGBM_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

clf1 = GridSearchCV(LGBMClassifier(random_state=0), LGBM_grid, cv=5, verbose=1)
clf1.fit(x_train, y_train)
clf1.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}

In [ ]:
# Best params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}
# clf_LGBM = LGBMClassifier(learning_rate = 0.05, max_depth=4, n_estimators=200, random_state=0).fit(x_train, y_train)

clf_LGBM = LGBMClassifier(**clf1.best_params_, random_state=0).fit(x_train, y_train)

train_predictions = clf_LGBM.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train LGBM: %.2f%%" % (accuracy * 100.0))

val_predictions = clf_LGBM.predict(x_val)
accuracy = accuracy_score(y_val, val_predictions)
print("Accuracy test LGBM: %.2f%%" % (accuracy * 100.0))
ConfusionMatrixDisplay.from_predictions(y_val, val_predictions)
plt.show()

print(classification_report(y_val, clf.predict(x_val)))

Accuracy train LGBM: 82.68%
Accuracy test LGBM: 81.73%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       778
           1       0.81      0.77      0.79       787

    accuracy                           0.79      1565
   macro avg       0.79      0.79      0.79      1565
weighted avg       0.79      0.79      0.79      1565



#### CatBoost

In [ ]:
clf2 = GridSearchCV(CatBoostClassifier(random_state=0, verbose=False), LGBM_grid, cv=5, verbose=1)
clf2.fit(x_train, y_train)
clf2.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}

In [ ]:
# Best params: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}
# clf_CatBoost = CatBoostClassifier(learning_rate = 0.1, max_depth=4, n_estimators=150, verbose=False, random_state=0).fit(x_train, y_train)

clf_CatBoost = CatBoostClassifier(**clf2.best_params_, verbose=False, random_state=0).fit(x_train, y_train)

train_predictions = clf_CatBoost.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train LGBM: %.2f%%" % (accuracy * 100.0))

val_predictions = clf_CatBoost.predict(x_val)
accuracy = accuracy_score(y_val, val_predictions)
print("Accuracy test LGBM: %.2f%%" % (accuracy * 100.0))
ConfusionMatrixDisplay.from_predictions(y_val, val_predictions)
plt.show()

print(classification_report(y_val, clf.predict(x_val)))

Accuracy train LGBM: 82.28%
Accuracy test LGBM: 81.28%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       778
           1       0.81      0.77      0.79       787

    accuracy                           0.79      1565
   macro avg       0.79      0.79      0.79      1565
weighted avg       0.79      0.79      0.79      1565



#### Random Forest

In [ ]:
RF_grid = { 
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,10],
    'criterion' :['gini', 'entropy']
}

In [ ]:
clf3 = GridSearchCV(RandomForestClassifier(random_state=0), RF_grid, cv=5, verbose=1)
clf3.fit(x_train, y_train)
clf3.best_params_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 100}

In [ ]:
# Best params: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 100}
# clf_RF = RandomForestClassifier(criterion='gini', max_depth=10, max_features='auto', n_estimators=100, verbose=0, random_state=0).fit(x_train, y_train)

clf_RF = RandomForestClassifier(**clf3.best_params_, verbose=0, random_state=0).fit(x_train, y_train)

train_predictions = clf_RF.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train LGBM: %.2f%%" % (accuracy * 100.0))

val_predictions = clf_RF.predict(x_val)
accuracy = accuracy_score(y_val, val_predictions)
print("Accuracy test LGBM: %.2f%%" % (accuracy * 100.0))
ConfusionMatrixDisplay.from_predictions(y_val, val_predictions)
plt.show()

print(classification_report(y_val, clf.predict(x_val)))

Accuracy train LGBM: 86.08%
Accuracy test LGBM: 80.70%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       778
           1       0.81      0.77      0.79       787

    accuracy                           0.79      1565
   macro avg       0.79      0.79      0.79      1565
weighted avg       0.79      0.79      0.79      1565



#### MLP

In [ ]:
MLP_grid = {
    'hidden_layer_sizes': [(20,),(30,)],
    'max_iter': [200],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'tol': [0.001]
}

In [ ]:
clf4 = GridSearchCV(MLPClassifier(random_state=0), MLP_grid, cv=5, verbose=1)
clf4.fit(x_train, y_train)
clf4.best_params_

Fitting 5 folds for each of 32 candidates, totalling 160 fits


{'activation': 'relu',
 'alpha': 0.05,
 'hidden_layer_sizes': (20,),
 'learning_rate': 'constant',
 'max_iter': 200,
 'solver': 'adam',
 'tol': 0.001}

In [ ]:
# Best params: {'activation': 'relu','alpha': 0.05,'hidden_layer_sizes': (20,),'learning_rate': 'constant','max_iter': 200,'solver': 'adam','tol': 0.001}
# clf_MLP = MLPClassifier(activation= 'relu',alpha= 0.05,hidden_layer_sizes= (20,),learning_rate= 'constant',max_iter= 200,solver= 'adam',tol= 0.001, random_state=0).fit(x_train, y_train)

clf_MLP = MLPClassifier(**clf4.best_params_, random_state=0).fit(x_train, y_train)

train_predictions = clf_MLP.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train LGBM: %.2f%%" % (accuracy * 100.0))

val_predictions = clf_MLP.predict(x_val)
accuracy = accuracy_score(y_val, val_predictions)
print("Accuracy test LGBM: %.2f%%" % (accuracy * 100.0))
ConfusionMatrixDisplay.from_predictions(y_val, val_predictions)
plt.show()

print(classification_report(y_val, clf.predict(x_val)))

Accuracy train LGBM: 81.16%
Accuracy test LGBM: 81.60%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       778
           1       0.81      0.77      0.79       787

    accuracy                           0.79      1565
   macro avg       0.79      0.79      0.79      1565
weighted avg       0.79      0.79      0.79      1565



In [ ]:
FOLDS=5
cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=10)

x_train2 = np.array(x_train)
y_train2 = np.array(y_train)

for clfi, name in zip([clf,clf_CatBoost,clf_LGBM,RidgeClassifier(random_state=0),XGBClassifier(random_state=0),clf_MLP,KNeighborsClassifier(),clf_RF],names):
    print(clfi)
    avg_accuracy = 0
    for fold, (train_idx, val_idx) in enumerate(cv.split(x_train2, y_train2)):
        xi, yi = x_train2[train_idx], y_train2[train_idx]
        x_valid, y_valid = x_train2[val_idx], y_train2[val_idx]
        clfi = clfi.fit(xi, yi)

        test_predictions = clfi.predict(x_valid)
        accuracy = accuracy_score(y_valid, test_predictions)
        avg_accuracy +=accuracy
        print(f"Precisión test fold {fold}: {accuracy * 100.0 :.2f}" % ())

    avg_accuracy /= FOLDS
    print(f'Avg. accuracy = {avg_accuracy * 100}')

DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=0)
Precisión test fold 0: 78.19
Precisión test fold 1: 75.64
Precisión test fold 2: 76.28
Precisión test fold 3: 76.02
Precisión test fold 4: 77.38
Avg. accuracy = 76.70182065210453
Precisión test fold 0: 80.27
Precisión test fold 1: 80.59
Precisión test fold 2: 79.95
Precisión test fold 3: 80.02
Precisión test fold 4: 80.10
Avg. accuracy = 80.18532139144915
LGBMClassifier(learning_rate=0.05, max_depth=4, n_estimators=200,
               random_state=0)
Precisión test fold 0: 81.23
Precisión test fold 1: 79.95
Precisión test fold 2: 79.79
Precisión test fold 3: 79.78
Precisión test fold 4: 79.70
Avg. accuracy = 80.08937259138375
RidgeClassifier(random_state=0)
Precisión test fold 0: 77.00
Precisión test fold 1: 75.48
Precisión test fold 2: 78.27
Precisión test fold 3: 76.58
Precisión test fold 4: 76.02
Avg. accuracy = 76.66974407694292
XGBClassifier()
Precisión test fold 0: 80.43
Precisión test fold 1: 79.39
Precisi

### Generar la salida para entregar

In [ ]:
#ya lo habia leido al principio
#test_df = pd.read_csv('test.csv')
#test_df = pd.read_csv('/kaggle/input/aprendizaje-supervisado-2022/test.csv')


Para poder evaluar nuestra predicción los datos de prueba deben tener exactamente el mismo tratamiento que los datos de entrenamiento

Generamos la salida

In [ ]:
clasificadores = [("CB", clf_CatBoost),
                  ("LGBM", clf_LGBM),
                  ("RF", clf_RF),
                  ("MLP", clf_MLP),
                  ("XGB", XGBClassifier(random_state=0))
                 ]

clasificadores_votantes = VotingClassifier(estimators=clasificadores, voting="soft")

In [ ]:
cv = StratifiedKFold(n_splits=5)
for train_index, val_index in cv.split(x_train, y_train):
    _x_train, _x_val = x_train.iloc[train_index], x_train.iloc[val_index]
    _y_train, _y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    clasificadores_votantes.fit(_x_train, _y_train)
    print(clasificadores_votantes.score(_x_val, _y_val)) 


0.7947284345047924
0.7971246006389776
0.7875399361022364
0.8169464428457234
0.8105515587529976


In [ ]:
#test_pred = clf.predict(test_X)
test_pred = clasificadores_votantes.predict(test_X)

In [ ]:
submission = pd.DataFrame(list(zip(test_id, test_pred)), columns=["PassengerId", "Transported"])
submission.to_csv("sample_submission.csv", header=True, index=False)

In [ ]:
!ls -l
!head sample_submission.csv
!wc -l sample_submission.csv

total 16
drwxr-xr-x 4 root root 4096 Sep 14 08:01 catboost_info
drwxr-xr-x 1 root root 4096 Sep  9 13:22 sample_data
-rw-r--r-- 1 root root 5999 Sep 14 08:54 sample_submission.csv
PassengerId,Transported
415,1
8663,1
4524,0
8246,0
2935,1
6195,0
5878,0
7897,1
5131,0
871 sample_submission.csv
